In [ ]:
import tensorflow as tf  
from tensorflow.examples.tutorials.mnist import input_data  
from tensorflow.contrib import rnn  
  
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

In [ ]:
#parameters
n_inputs = 28
max_time = 28  
lstm_size = 200
n_class = 10 
batch_size = 50
n_batch = mnist.train.num_examples // batch_size
  
x = tf.placeholder(tf.float32, [None, 784])  
y = tf.placeholder(tf.float32, [None, 10])  

In [ ]:
#Store weights & biases
weights = tf.Variable(tf.truncated_normal([lstm_size, n_class], stddev = 0.1))  
biases = tf.Variable(tf.constant(0.1, shape=[n_class]))  

In [ ]:
#LSTM
def LSTM(X, weights, biases):  
    #inputs format : [batch_size, max_time, n_inputs]  
    inputs = tf.reshape(X, [-1, max_time, n_inputs])  

    lstm_cell = rnn.BasicLSTMCell(lstm_size)  
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)  
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)  
    return results 

In [ ]:
#Construct the model
prediction = LSTM(x, weights, biases)

In [ ]:
#Define cross_entropy and optimizer
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))  
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [ ]:
#Evaluate the model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
#Launch the graph
with tf.Session() as session:  
    session.run(init)  
    for epoch in range(60):  
        for batch in range(batch_size):  
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})  
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})  
        print 'Iter' + epoch + 'accuracy: ' + acc
        
    print "Optimization Finished"
    #Calculate accuracy for the first 256 mnist images
    print "Testing accuracy: ", session.run(accuracy, feed_dict = {x: mnist.test.images[:256], y: mnist.test.labels[:256]})